# 🗺️Geospatial Analysis Course: Practical Guide😎
> __*On Bishkek Air Condition data*__

![Geospatial Analysis in Vrubel style](https://user-images.githubusercontent.com/115424463/282264815-cba13dc0-ac16-4bc6-8628-112876bd1364.jpeg)

> Picture: Geospatial Analysis in Vrubel style

## Hello, fellow data enthusiasts 🤓!

After completing the fantastic [__Geospatial Analysis__](https://www.kaggle.com/learn/geospatial-analysis) microcourse I decided to apply the tools and techniques to a new dataset and consolidate everything into one comprehensive notebook. I believe my work will help you grasp the theory and serve as a practical starting point.

**The Data**:
- I aggregated data from [__7 Air Quality Sensors in Bishkek__](https://maps.sensor.community/#12/42.8597/74.6035), Kyrgyzstan during November 11, 2023.
- I processed [__Administrative Boundaries of Kyrgyzstan__](https://data.humdata.org/dataset/cod-ab-kgz?) with GeoPandas to extract the shape of Bishkek. 


**The Map**:
- It is interactive and includes a 'layers control' where you can select the base map style and several features (__FeatureGroup__ , __TileLayer__) 
- Sensors. I combined __Circles__ and __MakeCluster__ in one feature. The color of these markers is determined by two parameters shown by the sensors: [__PM10__](https://www.epa.vic.gov.au/for-community/environmental-information/air-quality/pm10-particles-in-the-air) and [__PM2.5__](https://www.google.com/search?q=pm2.5+scale&oq=PM2.5+scale&gs_lcrp=EgZjaHJvbWUqBwgAEAAYgAQyBwgAEAAYgAQyCggBEAAYhgMYigUyCggCEAAYhgMYigUyCggDEAAYhgMYigXSAQg2MDQ1ajBqN6gCALACAA&sourceid=chrome&ie=UTF-8)
- Popups provide additional information about sensor values
- Bishkek Administrative boundaries with __Choropleth__
- MousePosition() displays the latitude and longitude of the cursor's position.
- Scroll restrictions, so that you don't end up studing Antarctica with my map.

__Project Scope__:
1. The data is taken for 1 day for demonstration purposes. To make it look like [this](https://maps.sensor.community/#12/42.8597/74.6035) some ETL should be considered.
2. Administrative boundaries of Bishkek do not look acurate. The avaliable [OCHA data](https://data.humdata.org/dataset/cod-ab-kgz?) contains boundaries for all administrative units of Kyrgyzstan except for the capital city. Consequently, I had to merge several polygons to create the shape you see. On the other hand, I didn't figure out how to extract the specific [Nominatim relation 8493930](https://nominatim.openstreetmap.org/ui/search.html?q=Bishkek).<br> **If you have insights on how to do that, please share in the comments**

<blockquote style="margin-right:auto; margin-left:auto; background-color: #faf0be; padding: 1em; margin:24px;">
<strong> If you find this project valuable, please upvote it, and feel free to provide feedback or suggestions in the comments. </strong> <br> 
    </blockquote>


In [1]:
import geopandas as gpd
import pandas as pd
import folium
from folium import Choropleth, Circle, Marker, FeatureGroup
from folium.plugins import MarkerCluster, MousePosition
import math

from warnings import simplefilter
simplefilter("ignore")

In [2]:
sensors_df = pd.read_csv('/kaggle/input/2023-11-10-bishkek-air-condition/2023_11_10_Bishkek_Air.csv')
sensors = gpd.GeoDataFrame(sensors_df, geometry=gpd.points_from_xy(sensors_df.lon, sensors_df.lat))
sensors.crs = {'init': 'epsg:4326'}
sensors

,sensor_id,lat,lon,P1,P2,geometry
0,33016,42.923081,74.605850,26.186886,10.386557,POINT (74.60585 42.92308)
1,34313,42.885263,74.554349,142.125105,43.713049,POINT (74.55435 42.88526)
2,35745,42.812000,74.628000,14.905766,12.203305,POINT (74.62800 42.81200)
3,52798,42.872000,74.622000,33.798790,16.510647,POINT (74.62200 42.87200)
4,55837,42.850056,74.632970,5.975216,5.975216,POINT (74.63297 42.85006)
5,76617,42.816000,74.648000,25.710925,12.383973,POINT (74.64800 42.81600)
6,83895,42.812000,74.638000,24.906562,10.252933,POINT (74.63800 42.81200)


In [3]:
# Find Bishkek's geolocation
world_filepath = gpd.datasets.get_path('naturalearth_cities')
world = gpd.read_file(world_filepath)
world.query('name == "Bishkek"')

,name,geometry
85,Bishkek,POINT (74.58326 42.87503)


In [4]:
# Create the base map with center in Bishkek and scrolling limits
m = folium.Map(location=[42.87503,74.58326],
                max_bounds=True,
                min_lat= 42.7,
                max_lat= 43,
                min_lon= 74.4,
                max_lon= 74.8,
               zoom_start=13)

# Add a second map style option for the initial map view
folium.TileLayer('Cartodb Positron').add_to(m)
MousePosition().add_to(m)

# Add Bishkek Administrative Boundaries as a separate group for layers control 
Boundaries = FeatureGroup("Bishkek Administrative").add_to(m)
kgz = gpd.read_file('/kaggle/input/2023-11-10-bishkek-air-condition/kgz_administrative.geojson')

Choropleth(
    geo_data=kgz.__geo_interface__,
    name="boundary",
    fill_color="#99A8B1",
    fill_opacity=0.4,
    line_opacity=0.2,
    legend_name="Administrative boundaries of Bishkek",
).add_to(Boundaries)


# Add Sensors as a group for the layers control
Sensors = FeatureGroup("Sensors").add_to(m)

def color_producer_PM10 (val):
    if val <= 40:
        return 'forestgreen'
    elif val <= 80:
        return 'yellow'
    elif val <= 120:
        return 'orange'    
    else:
        return 'darkred'

# Clusters and Circles in one feature
mc = MarkerCluster().add_to(Sensors)
for idx, row in sensors.iterrows():
    if not math.isnan(row['lon']) and not math.isnan(row['lat']):
        mc.add_child(
                    Circle(
                        location = [row['lat'], row['lon']],
                        radius=1000,
                        color = color_producer_PM10(row['P1']),
                        fill_opacity=0.7,
                        fill_color=color_producer_PM10(row['P1']),
                        tooltip=f"PM10 = {int(row.P1)}, PM2.5 = {int(row.P2)}"
                    )
        )

# Add layers control
folium.LayerControl().add_to(m)
        
# Display the map
m

**Credits**:
- [Alexis Cook](https://www.kaggle.com/alexisbcook) and [Jessica Li](https://www.kaggle.com/jessicali9530) for the [Geospatial Analysis course](https://www.kaggle.com/learn/geospatial-analysis)
- [Pavel Isayenko](https://www.kaggle.com/pavelisayenko) for his game-changing tip on where to retrieve the Air Quality Sensors data.

If you liked this notebook, please also check the others:
 - [🕒Time Series Course: A Practical Guide](https://www.kaggle.com/code/ivanlydkin/time-series-course-a-practical-guide)
 - [🤖Computer Vision course: Practical Guide](https://www.kaggle.com/code/ivanlydkin/computer-vision-course-practical-guide)
 - [🛳️Original feature for Titanic](https://www.kaggle.com/code/ivanlydkin/titanic-case-with-some-original-features)


*I hope it was helpful* 🤝<br>
<blockquote style="margin-right:auto; margin-left:auto; background-color: #faf0be; padding: 1em; margin:24px;">
<strong> If you find this project valuable, please upvote it, and feel free to provide feedback or suggestions in the comments.</strong> </blockquote><br> 